In [15]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from sklearn.model_selection import train_test_split
from mSDA.msda_theano import mSDATheano
import theano.tensor as T

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[1]

# Pruebas con el dataset Amazon (3000 Dimensiones)

In [2]:
dims = 3000
dataset_name = datasets[0]

In [12]:
print tipo
print dataset_name
print dims
print data_path

msda
amazon
3000
data


In [13]:
# cargando dataset
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [14]:
parametros = {
    'noises': [0.3, 0.5, 0.8],
    'layers': [1, 3, 5]
}

noise_layer_pair =  list(itertools.product(parametros['noises'], parametros['layers']))

x = T.dmatrix('x')

In [ ]:
df = pd.DataFrame(columns=dataframe_columns+'Porcentaje')

porcentajes = [25, 50, 100]

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

for src, tgt in pairs:
    #datos de entrenamiento
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
    
    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tgt = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
    
    X_train = np.concatenate((X_src, X_tgt))
    
    #datos de prueba
    X_tgt_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_tgt_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
    
    
    #baseline in-domain error
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)
    
    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tgt, y_tgt)
    b_error = 1-svc.score(X_tgt_ts, y_tgt_ts)
    
    
    for noise, layer in noise_layer_pair:
        # entrenar un modelo de adaptacion
        modelo_temp = mSDATheano(x, layer, noise)
        t_adaptar = modelo_temp.fit(X_train)
        
        # entrenar un clasificador con los datos adaptados
        # X_src_trans = modelo_temp.predict(X_src)
        # clf = svc.fit
        
        # se prueba con distintos porcentajes
        
    